In [52]:
import yfinance as yf

# Define ticker and date range
ticker = "QQQ"
start_date = "2024-09-05"
end_date = "2024-09-30"

# Download historical data with 5-minute interval
qqq_data = yf.download(ticker, start=start_date, end=end_date, interval="5m")

# Save to CSV
qqq_data.to_csv("QQQ_2023_data.csv")


[*********************100%***********************]  1 of 1 completed


In [53]:
import pandas as pd

# Parameters
starting_balance = 100000  # Starting balance for backtesting
balance = starting_balance
results = []  # Store trade results

# Load historical data
# Assume CSV format: Price,Adj Close,Close,High,Low,Open,Volume
historical_data = pd.read_csv('QQQ_2023_data.csv', skiprows=3)  # 첫 두 줄을 건너뜀

# Fix the column names if necessary
historical_data.columns = ['Datetime', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
print(historical_data)
# # Convert the 'Datetime' column to datetime type
historical_data['Datetime'] = pd.to_datetime(historical_data['Datetime'])

# # Sort data by datetime (if not already sorted)
historical_data = historical_data.sort_values(by='Datetime')

positions = {}
first_five_min_candle = None

# Further processing (e.g., implementing trading strategy) would go here


                       Datetime   Adj Close       Close        High  \
0     2024-09-05 13:35:00+00:00  461.709991  461.709991  462.549591   
1     2024-09-05 13:40:00+00:00  461.640015  461.640015  462.179993   
2     2024-09-05 13:45:00+00:00  461.470001  461.470001  462.653900   
3     2024-09-05 13:50:00+00:00  462.619995  462.619995  462.619995   
4     2024-09-05 13:55:00+00:00  463.049988  463.049988  463.190002   
...                         ...         ...         ...         ...   
1320  2024-09-27 19:35:00+00:00  486.940002  486.940002  487.130005   
1321  2024-09-27 19:40:00+00:00  486.769989  486.769989  486.959991   
1322  2024-09-27 19:45:00+00:00  486.559998  486.559998  486.890015   
1323  2024-09-27 19:50:00+00:00  487.130005  487.130005  487.269989   
1324  2024-09-27 19:55:00+00:00  486.779999  486.779999  487.349915   

             Low        Open   Volume  
0     459.549988  459.660004   979262  
1     461.329987  461.730011   463345  
2     460.850006  461.63000

In [63]:
startTime = '13:35'
endTime = '19:55'

daily_results = []  # Store results for each day

current_day = None

# Function to simulate ORB strategy on historical data
def backtest_orb_strategy(df, qty=10000):
    global balance, first_five_min_candle, current_day
    first_five_min_candle = None

    for _, row in df.iterrows():
        current_time = row['Datetime']
        open_price = row['Open']
        high_price = row['High']
        low_price = row['Low']
        close_price = row['Close']
        
        # Check if the day has changed
        if current_day != current_time.date():
            # Log end-of-day results for the previous day
            if current_day is not None:
                
                daily_results.append({
                    "date": current_day,
                    "balance": balance,
                    "profit_loss": balance - starting_balance
                })

            # Reset for the new day
            current_day = current_time.date()
            first_five_min_candle = None
            positions.clear()  # Clear positions at the start of each day

        # Define first 5-minute candle only once at 13:35
        if current_time.time() == pd.Timestamp(startTime).time() and first_five_min_candle is None:
            first_five_min_candle = {'open': open_price, 'close': close_price, 'high': high_price, 'low': low_price}

        # Execute ORB strategy after defining first 5-minute candle
        if first_five_min_candle:
            # QQQ is bullish, buy TQQQ (3x leveraged)
            if first_five_min_candle['close'] > first_five_min_candle['open'] and 'TQQQ' not in positions:
                positions['TQQQ'] = {'buy_price': close_price, 'qty': qty}
                print(f"Bought TQQQ at {close_price}")
            
            # QQQ is bearish, buy SQQQ (3x inverse leveraged)
            elif first_five_min_candle['close'] < first_five_min_candle['open'] and 'SQQQ' not in positions:
                positions['SQQQ'] = {'buy_price': close_price, 'qty': qty}
                print(f"Bought SQQQ at {close_price}")
            
            # Check take-profit and stop-loss
            monitor_backtest_positions(row['Datetime'], close_price)
        
        # Only close positions at 19:55 if first_five_min_candle is set and there are open positions
        if current_time.time() == pd.Timestamp(endTime).time() and first_five_min_candle is not None and positions:
            close_all_positions(close_price, current_time)

# Monitor positions with take-profit and stop-loss during backtesting
def monitor_backtest_positions(current_time, current_price):
    global balance
    for symbol, data in list(positions.items()):
        buy_price = data['buy_price']
        qty = data['qty']
        
        # Adjust current price for leverage
        if symbol == 'TQQQ':
            adjusted_price = buy_price + 3 * (current_price - buy_price)  # 3x leverage in the same direction
        elif symbol == 'SQQQ':
            adjusted_price = buy_price - 3 * (current_price - buy_price)  # 3x leverage in opposite direction

        # Take-profit
        if adjusted_price >= buy_price * 1.1:
            balance += (adjusted_price - buy_price) * qty
            print(f"Take profit: Sold {symbol} at {adjusted_price} (1.5x buy price)")
            del positions[symbol]

        # Stop-loss
        elif adjusted_price <= buy_price * 0.95:
            balance -= (buy_price - adjusted_price) * qty
            print(f"Stop loss: Sold {symbol} at {adjusted_price} (below 95% of buy price)")
            del positions[symbol]

# Close all remaining positions at 19:55
def close_all_positions(close_price, current_time):
    global balance
    if not positions:  # Check if there are no open positions
        return

    for symbol, data in list(positions.items()):
        buy_price = data['buy_price']
        qty = data['qty']
        
        # Calculate final profit/loss at close price
        if symbol == 'TQQQ':
            final_price = buy_price + 3 * (close_price - buy_price)
        elif symbol == 'SQQQ':
            final_price = buy_price - 3 * (close_price - buy_price)

        balance += (final_price - buy_price) * qty
        print(f"End of day close at {endTime}: Sold {symbol} at {final_price}")
        del positions[symbol]

# Run backtesting
backtest_orb_strategy(historical_data)

# Log final day results
daily_results.append({
    "date": current_day,
    "balance": balance,
    "profit_loss": balance - starting_balance
})

# Show results for each day
for result in daily_results:
    print(f"Date: {result['date']}, Balance: {result['balance']}, Profit/Loss: {result['profit_loss']}")

print(f"Starting Balance: {starting_balance}")
print(f"Ending Balance: {balance}")
print(f"Total Profit/Loss: {balance - starting_balance}")



Bought TQQQ at 461.7099914550781
End of day close at 19:55: Sold TQQQ at 460.00006103515625
Bought SQQQ at 457.5299987792969
End of day close at 19:55: Sold SQQQ at 483.510009765625
Bought TQQQ at 453.7999877929688
End of day close at 19:55: Sold TQQQ at 455.9300537109374
Bought SQQQ at 455.5799865722656
End of day close at 19:55: Sold SQQQ at 446.3099060058594
Bought SQQQ at 458.1311950683594
Stop loss: Sold SQQQ at 434.3748168945311 (below 95% of buy price)
Bought SQQQ at 465.6700134277344
End of day close at 19:55: Sold SQQQ at 457.3300170898436
Bought SQQQ at 468.5799865722656
End of day close at 19:55: Sold SQQQ at 454.5999755859375
Bought TQQQ at 474.010009765625
End of day close at 19:55: Sold TQQQ at 478.1199951171875
Bought TQQQ at 471.3900146484375
End of day close at 19:55: Sold TQQQ at 476.8799743652344
Bought TQQQ at 475.8999938964844
End of day close at 19:55: Sold TQQQ at 468.73004150390625
Bought SQQQ at 473.510009765625
End of day close at 19:55: Sold SQQQ at 480.08001